# Setup

In [14]:
import os
HOME = os.getcwd()

In [15]:
import openvino as ov
import torch
import torchvision

TORCH_VERSION = ".".join(torch.__version__.split(".")[:2])
CUDA_VERSION = torch.__version__.split("+")[-1]

Install YoloV8

In [16]:
!pip install "ultralytics<=8.3.40"

from IPython import display
display.clear_output()

import ultralytics
ultralytics.checks()

Ultralytics 8.3.40  Python-3.12.7 torch-2.5.1 CPU (11th Gen Intel Core(TM) i7-1165G7 2.80GHz)
Setup complete  (8 CPUs, 15.7 GB RAM, 69.8/476.2 GB disk)


Install YoloV5

In [17]:
%cd {HOME}
!git clone https://github.com/ultralytics/yolov5

%cd {HOME}/yolov5
!pip install -r requirements.txt

from IPython import display
display.clear_output()

Install Detectron2

In [ ]:
from IPython import display
display.clear_output()

import detectron2
